## **Introducción**

Este trabajo aborda un problema de **clasificación de texto clínico en español**: predecir la **gravedad** del caso (leve, moderado, severo) a partir de notas clínicas breves y la afección reportada. El objetivo es **construir y comparar** dos enfoques de NLP —uno clásico con **embeddings** y otro con **transformers**— para entregar un sistema **asistivo** (no diagnóstico), reproducible y evaluado con criterios justos entre subgrupos.

---

### **Enfoque metodológico**

* **Preprocesamiento** (**solo para embeddings**): tokenización, **lematización** y e**liminación de stopwords** con spaCy.

 *El pipeline transformer usa texto crudo para respetar el preentrenamiento*.

* **Pipeline de Embeddings**:

  * **Word2Vec** (**40d**) entrenado en train; vector oracional por promedio.

  * **Features tabulares**: género binario (0/1) y edad estandarizada.

  * **Clasificador**: **Random Forest**.

* **Pipeline Transformer**:

  * **BETO** (`dccuchile/bert-base-spanish-wwm-uncased`) con **Keras/TensorFlow**.

  * Fine-tuning breve: **2–3 épocas**, `lr≈2e-5`, `max_length≤256`, *batch* moderado, **EarlyStopping** simple.

* **Evaluación**: *train/test split* estratificado (80/20). **Precision, recall, F1** (macro y por clase) + **matriz de confusión** para ambos pipelines.

* **Análisis de sesgos (fairness)**: desempeño estratificado por **género** y por **rangos etarios** (`<30, 30–50, >50`) y **propuestas de mitigación** si hay brechas.

* **Ética y uso responsable**: privacidad, sesgos del lenguaje clínico y carácter estrictamente **asistivo** del modelo.

* **Reproducibilidad**: semillas fijadas (`random`, `numpy`, `tf`) y entrenamiento de **Word2Vec**.

---

### **Variables del dataset**

* `texto_clinico`: nota breve en lenguaje natural (español).

* `afeccion`: diagnóstico o afección reportada (texto corto).

* `gravedad`: etiqueta objetivo con tres clases (leve, moderado, severo).

* `genero`: variable categórica (mapeada a binaria para el modelo clásico).

* `edad`: variable numérica (posteriormente estandarizada).

El propósito es comparar un baseline robusto (**Word2Vec+RF**) con un **transformer** en español (**BETO**), medir **calidad predictiva** y **comportamiento por subgrupos**, y cerrar con una **reflexión ética** alineada al uso responsable de NLP en contextos clínicos.

### PUNTO 1 — **Preparación**



In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # obliga a usar tf.keras

import random, numpy as np, tensorflow as tf
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)

import pandas as pd
from sklearn.model_selection import train_test_split

Carga y columnas base (concatenar textos)

In [ ]:
df = pd.read_csv("dataset_clinico_simulado_200.csv", encoding="utf-8")


#### **Preprocesamiento**

In [ ]:
import re

# limpiar 'afeccion' y reconstruir 'texto'
def limpiar_afeccion(s):
    s = re.sub(r"\b(?:leve|moderado|severo)\b", "", str(s), flags=re.I)
    s = re.sub(r"\s+", " ", s).strip()
    return s

# si hay NaN en 'afeccion', evita problemas
df["afeccion"] = df["afeccion"].fillna("")

# limpiar 'afeccion' para quitar leve|moderado|severo
df["afeccion_clean"] = df["afeccion"].apply(limpiar_afeccion)

# construir el texto final que usa el modelo
df["texto"] = (
    df["texto_clinico"].astype(str).str.strip() + " " + df["afeccion_clean"]
).str.strip()


In [ ]:
# Sanity check: que 'afeccion_clean' ya NO tenga leve/moderado/severo
pat = re.compile(r"\b(?:leve|moderado|severo)\b", re.I)
mask = df["afeccion_clean"].str.contains(pat, na=False)
assert not mask.any()  # si falla, quedan casos por limpiar

# (opcional) si falla, inspecciona cuáles:
# df.loc[mask, ["afeccion", "afeccion_clean"]].head()


In [ ]:
# Features y target
X = df[["texto", "genero", "edad"]].copy()   # solo predictores
y = df["gravedad"].astype(str)               # etiqueta a predecir

print(df.head())

                                       texto_clinico  edad genero  \
0  El paciente presenta síntomas leves como dolor...    43      M   
1  Consulta por hipertensión leve sin signos de a...    34      F   
2  Evolución rápida del cuadro clínico, compatibl...    34      M   
3  El paciente presenta síntomas leves como tos y...    58      M   
4  El cuadro clínico indica asma, con síntomas co...    45      M   

                     afeccion  gravedad              afeccion_clean  \
0             resfriado común      leve             resfriado común   
1           hipertensión leve      leve                hipertensión   
2  infarto agudo de miocardio    severo  infarto agudo de miocardio   
3             gastroenteritis      leve             gastroenteritis   
4                        asma  moderado                        asma   

                                               texto  
0  El paciente presenta síntomas leves como dolor...  
1  Consulta por hipertensión leve sin signos de 

Se decidió **concatenar las columnas `texto_clinico` y `afeccion`** en una nueva columna llamada `texto` con el objetivo de **enriquecer la representación semántica** que recibirán los modelos de NLP.  

- La columna `texto_clinico` contiene la descripción narrativa de síntomas y observaciones del paciente.  
- La columna `afeccion` aporta el diagnóstico o la condición clínica asociada.  

Al unir ambas fuentes de información en un solo campo:
1. Se aumenta la **densidad de señal** en cada registro, ya que el modelo dispone de más contexto para identificar patrones relacionados con la gravedad clínica.  
2. Se reduce el riesgo de que la afección quede aislada como metadato y no sea considerada dentro del análisis textual.  
3. Se genera un **corpus más robusto** para los métodos de representación (Word2Vec, embeddings), lo cual es especialmente importante dado el tamaño reducido del dataset (200 registros).  

Esta decisión metodológica busca que el modelo aprenda no solo de los síntomas descritos, sino también de la afección asociada, incrementando la capacidad de detección de la **gravedad clínica** en el proceso de clasificación.

---


#### Split estratificado

Para medir la Precision/Recall/F1 (macro y por clase).

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED, stratify=y
)

len(X_train), len(X_test), y_train.value_counts().to_dict()


(160, 40, {'moderado': 66, 'severo': 47, 'leve': 47})

Se separó el dataset en un **80% para entrenamiento** y un **20% para prueba**.  
Se utilizó `stratify=y` para mantener la misma proporción de clases en ambas particiones de los datos.  

Esto es importante en clasificación de textos porque asegura que todas las categorías de **gravedad clínica** (leve, moderado, severo) estén bien representadas.  

La distribución de clases en el conjunto de entrenamiento quedó de la siguiente manera:  
- moderado: 66 casos  
- severo: 47 casos  
- leve: 47 casos  

Esto confirma que la estratificación funcionó correctamente y que se mantiene un **equilibrio inicial entre las clases**.  
Este equilibrio es útil porque permitirá observar de manera más clara cómo el modelo aprende a distinguir entre los diferentes niveles de gravedad clínica.

---


### PUNTO 2 — **Modelado**

Se comienza la construcción del primer pipeline de modelado.  
La idea es utilizar **Word2Vec** para transformar los textos clínicos en representaciones numéricas (vectores), y luego entrenar un **Random Forest** como clasificador.

1. **Instalación y librerías**  
   - Se instaló la librería `gensim`, que es ampliamente utilizada en NLP para entrenar modelos de embeddings.  
   - Se importó `Word2Vec` junto con `re` para el manejo de expresiones regulares necesarias en la tokenización.

2.  
   - Con Word2Vec se busca que las palabras de los textos clínicos queden representadas en un **espacio vectorial**, donde palabras con contextos similares queden más cercanas entre sí.  
   - Este es un avance respecto a representaciones más simples como Bag of Words o TF-IDF, ya que captura información semántica.  

   ---

#### Lematización + stopwords [solo embeddings]

* Se usa para el pipeline W2V+RF; BETO va con texto crudo.

In [ ]:
# Setup spaCy (1 sola vez)
#!pip -q install spacy==3.7.4
#!python -m spacy download es_core_news_sm

import spacy
nlp = spacy.load("es_core_news_sm", disable=["ner","parser","textcat","senter"])
STOP_ES = nlp.Defaults.stop_words


Se inicializa **spaCy** para español cargando el modelo `es_core_news_sm`, desactivando componentes que no se usan en este pipeline (**NER**, **parser**, **textcat** y **senter**) para acelerar el procesamiento.

Luego, se obtiene el conjunto de **stopwords en español** desde `nlp.Defaults.stop_words`, que se utilizará para **filtrar palabras vacías** durante la tokenización/lematización del pipeline de **embeddings** (**W2V+RF**).

#### Embeddings (Word2Vec 40d) + RandomForest

(tokenización → entrenar Word2Vec → vector promedio → agregar género/edad → RF → métricas)



In [ ]:
#!pip install gensim


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.6/26.6 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.1
    Uninstalling scipy-1.16.1:
      Successfully uninstalled scipy-1.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.1 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.


In [ ]:
from gensim.models import Word2Vec

In [ ]:
def simple_tokens(s: str):
    doc = nlp(str(s).lower())

    # lemas + sin stopwords + solo alfabéticos
    return [t.lemma_ for t in doc if t.is_alpha and (t.lemma_ not in STOP_ES)]

train_tokens = [simple_tokens(t) for t in X_train["texto"]]
test_tokens  = [simple_tokens(t) for t in X_test["texto"]]



In [ ]:
# Revisión rápida de tokens (lo que realmente verá W2V)
ex_idx = X_train.index[:3]  # 3 filas
tok_preview = []
for i in ex_idx:
    raw = X_train.loc[i, "texto"]    # o "texto_w2v"
    toks = simple_tokens(raw)
    tok_preview.append({"id": i, "raw": raw[:140]+"...", "tokens": toks[:20]})
import pandas as pd
display(pd.DataFrame(tok_preview))


,id,raw,tokens
0,191,"El cuadro clínico indica dolor lumbar, con sín...","[cuadro, clínico, indicar, dolor, lumbar, sínt..."
1,2,"Evolución rápida del cuadro clínico, compatibl...","[evolución, rápido, cuadro, clínico, compatibl..."
2,125,"El cuadro clínico indica dolor lumbar, con sín...","[cuadro, clínico, indicar, dolor, lumbar, sínt..."


In [ ]:
w2v = Word2Vec(
    sentences=train_tokens,
    vector_size=40, window=5, min_count=1, sg=1, workers=4, seed=SEED
)

1. **Función de tokenización (`simple_tokens`)**  
   Se definió una función que extrae palabras usando una expresión regular, manteniendo caracteres con acentos y la letra ñ, y transformando todo a minúsculas.  
   Esto permite que los textos clínicos queden divididos en **tokens limpios y homogéneos**, listos para ser procesados por el modelo de embeddings.

2. **Generación de tokens para entrenamiento y prueba**  
   - `train_tokens`: se tokenizó cada texto del conjunto de entrenamiento.  
   - `test_tokens`: se aplicó el mismo procedimiento al conjunto de prueba.  
   De esta forma, los modelos posteriores trabajarán con listas de palabras en lugar de cadenas de texto.

3. **Entrenamiento de Word2Vec**  
   Se entrenó un modelo Word2Vec con las siguientes configuraciones:  
   - `vector_size=40`: cada palabra se representa con un vector de 40 dimensiones.  
   - `window=5`: contexto de 5 palabras a la izquierda y derecha para aprender relaciones.  
   - `min_count=1`: se incluyen todas las palabras, incluso las que aparecen una sola vez (útil dado el tamaño reducido del dataset).  
   - `sg=1`: se usa **Skip-Gram**, un enfoque más efectivo con datasets pequeños para aprender buenas representaciones semánticas.  
   - `workers=4`: paralelización para acelerar el entrenamiento.  
   - `seed=SEED`: reproducibilidad de resultados.

---


#### Promedio de palabras → vector de oración (40d)

In [ ]:
def sent_vec(tokens, model, dim=40):
    # Se crea una lista con los vectores de cada palabra que esté en el vocabulario entrenado
    v = [model.wv[w] for w in tokens if w in model.wv]

    # Si la lista queda vacía (palabras fuera del vocabulario), se devuelve un vector de ceros
    if not v:
        return np.zeros(dim, dtype=np.float32)

    # Se calcula el promedio de los vectores de las palabras → vector de la oración
    return np.mean(v, axis=0).astype(np.float32)

# Se generan los embeddings promedio para cada oración del conjunto de entrenamiento
Xtr_emb = np.vstack([sent_vec(ts, w2v, 40) for ts in train_tokens])

# Se hace lo mismo para el conjunto de prueba
Xte_emb = np.vstack([sent_vec(ts, w2v, 40) for ts in test_tokens])

# Se revisan las dimensiones: 160 oraciones (train), 40 (test), cada una con vectores de 40 dimensiones
Xtr_emb.shape, Xte_emb.shape


((160, 40), (40, 40))

Se creó una función para convertir cada lista de tokens en un único vector de 40 dimensiones, calculando el **promedio de los embeddings de sus palabras**.  
De esta manera, cada nota clínica pasa de ser un conjunto de palabras a una representación numérica fija, lo que permite usarla después en el clasificador.  


#### Incorporación de variables demográficas (género binarizado y edad normalizada) → Representación final de 42 dimensiones


In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
# Mapear género a binario (F→0, M→1) y dejarlo como columna (n,1) para poder concatenar
gen_map = {"F":0, "M":1}
g_tr = X_train["genero"].map(gen_map).values.reshape(-1,1)
g_te = X_test["genero"].map(gen_map).values.reshape(-1,1)

# Estandarizar la edad con StandardScaler
# Ajustar (fit) SOLO con train para evitar fuga de información
sc_age = StandardScaler()
a_tr = sc_age.fit_transform(X_train[["edad"]])
a_te = sc_age.transform(X_test[["edad"]])

# Concatenar features finales:
# [40 dims de embeddings] + [1 dim género] + [1 dim edad] = 42 dims
Xtr_42 = np.hstack([Xtr_emb, g_tr, a_tr])  # (n_train, 42)
Xte_42 = np.hstack([Xte_emb, g_te, a_te])  # (n_test, 42)
Xtr_42.shape, Xte_42.shape

# Chequeo rápido de dimensiones esperadas
print(Xtr_42.shape, Xte_42.shape)

(160, 42) (40, 42)


En este paso se incorporaron dos variables adicionales al vector de texto (40 dimensiones de Word2Vec):

* Género → se codificó como binario (F=0, M=1).

* Edad → se normalizó con z-score usando StandardScaler.

Luego, estos valores se concatenaron con los embeddings, generando una representación final de 42 dimensiones para cada muestra.
Esto permite que el clasificador no solo aprenda de las palabras, sino también de factores demográficos relevantes, que más adelante se usarán para evaluar sesgos.

---

#### **Entrenamiento con Random Forest**

Para este dataset clínico de 200 registros, se eligió **Random Forest** como modelo de clasificación debido a varias razones:

1. **Robustez con datasets pequeños**:  
   Random Forest funciona bien incluso con un número limitado de ejemplos, evitando el sobreajuste gracias a la combinación de múltiples árboles de decisión.

2. **Capacidad de manejar variables mixtas**:  
   Además de los embeddings de texto, el modelo integra de forma natural variables adicionales como género (binario) y edad (numérica normalizada).

3. **Estabilidad y facilidad de interpretación**:  
   Comparado con modelos más complejos, Random Forest entrega resultados estables y métricas consistentes, lo cual es adecuado en un escenario de simulación con pocos datos.

4.   
   En esta etapa, el interés principal es mostrar cómo los textos clínicos pueden representarse con embeddings y usarse en un clasificador sólido. Random Forest cumple este rol sin requerir ajustes extensivos de hiperparámetros.


In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
rf = RandomForestClassifier(
    n_estimators=400, random_state=SEED, class_weight="balanced_subsample"
)
rf.fit(Xtr_42, y_train)


RandomForestClassifier(class_weight='balanced_subsample', n_estimators=400,
                       random_state=42)

#### Métricas de clasificación (macro y por clase)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
pred_rf = rf.predict(Xte_42)

print("== Embeddings40 + (genero, edad) | RandomForest ==")
print(classification_report(y_test, pred_rf, digits=3))
print(confusion_matrix(y_test, pred_rf))


== Embeddings40 + (genero, edad) | RandomForest ==
              precision    recall  f1-score   support

        leve      1.000     1.000     1.000        12
    moderado      1.000     1.000     1.000        16
      severo      1.000     1.000     1.000        12

    accuracy                          1.000        40
   macro avg      1.000     1.000     1.000        40
weighted avg      1.000     1.000     1.000        40

[[12  0  0]
 [ 0 16  0]
 [ 0  0 12]]


El modelo obtuvo **1.000** en precisión, recall y F1 para **todas** las clases (matriz de confusión sin errores).  
**Lectura:** con 200 filas y texto simulado, un 100% sugiere que el **texto clínico contiene señales muy directas de la gravedad** o patrones altamente separables.  

**Uso:** lo tomo como **baseline alto**; no implica generalización. Más adelante se verificará con BETO y se propondrá una **ablación** (ablation study; remover “leve/moderado/severo” del `texto_clinico` o detectar duplicados entre train/test) para medir sensibilidad del modelo.

---


####  **BERT (BETO) con Keras/TF**

Se eligió trabajar con **BERT (BETO, versión entrenada para español)** utilizando Keras/TensorFlow porque:

- **Modelo preentrenado**: BETO ya fue entrenado con grandes corpus en español, lo que permite aprovechar representaciones lingüísticas de alta calidad sin necesidad de entrenar desde cero.

- **Adecuado para clasificación de texto**: BERT está diseñado para capturar relaciones contextuales entre palabras, mejorando frente a técnicas más simples como promediar embeddings.  

- **Integración con Keras/TensorFlow**: usarlo dentro de este framework facilita construir un pipeline de clasificación, entrenar con pocas líneas de código y aprovechar GPU para acelerar el proceso.  

BETO actúa como un **modelo avanzado de NLP**, comparado con Random Forest + Word2Vec, permitiendo contrastar el desempeño de un enfoque tradicional con el de un transformer moderno.

#### Semilla TF e instalación

In [ ]:
import tensorflow as tf

In [ ]:
# fija la semilla en TensorFlow para garantizar reproducibilidad en los resultados
tf.random.set_seed(SEED)

#### Tokenizer + datasets (tf.data)

In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

In [ ]:
# Se define el modelo BETO (BERT para español) y su tokenizador
model_name = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Diccionarios de etiquetas ↔ índices
labels   = sorted(y.unique().tolist())
id2label = {i: l for i, l in enumerate(labels)}
label2id = {l: i for i, l in id2label.items()}

# Función para codificar lotes de texto (tokenización + labels)
def encode_batch(texts, ys):
    # Tokeniza con padding y truncado a longitud fija (256)
    enc = tokenizer(texts, padding=True, truncation=True, max_length=256)
    # Agrega etiquetas como enteros
    enc["labels"] = [label2id[v] for v in ys]
    return enc

# Datasets de entrenamiento y prueba (formato HuggingFace)
ds_tr = Dataset.from_dict(encode_batch(X_train["texto"].tolist(), y_train.tolist()))
ds_te = Dataset.from_dict(encode_batch(X_test["texto"].tolist(),  y_test.tolist()))

# --- Punto 1 (exploratorio): ver cómo tokeniza BETO un texto clínico ---
ejemplo = "El paciente presenta dolor torácico y disnea leve"
tok = tokenizer(ejemplo, padding=True, truncation=True, max_length=256)
print("input_ids:", tok["input_ids"])
print("attention_mask:", tok["attention_mask"])
print("tokens:", tokenizer.convert_ids_to_tokens(tok["input_ids"]))
# ----------------------------------------------------------------------

# Conversión a tf.data (lo que Keras/TF usará en el fit)
tf_tr = ds_tr.to_tf_dataset(
    columns=["input_ids", "attention_mask"],  # entradas que requiere BERT
    label_cols=["labels"],
    shuffle=True,
    batch_size=16
)

tf_te = ds_te.to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["labels"],
    shuffle=False,
    batch_size=32
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

input_ids: [4, 1039, 6420, 4167, 4141, 28856, 18721, 1040, 1218, 23080, 19890, 5]
attention_mask: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
tokens: ['[CLS]', 'el', 'paciente', 'presenta', 'dolor', 'torá', '##cico', 'y', 'dis', '##nea', 'leve', '[SEP]']


/usr/local/lib/python3.12/dist-packages/datasets/arrow_dataset.py:403: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


Se prepararon los textos clínicos para entrenar un modelo BETO con Keras/TensorFlow:

1. **Tokenización**: se utilizó el tokenizador propio de BETO para transformar cada texto en secuencias de enteros (`input_ids`) y máscaras de atención (`attention_mask`), asegurando un tamaño máximo de 256 tokens.  
2. **Etiquetas**: se codificaron las clases de gravedad clínica como enteros (`label2id`).  
3. **Datasets**: se crearon datasets de entrenamiento y prueba (`ds_tr`, `ds_te`), y luego se pasaron al formato de TensorFlow (`tf_tr`, `tf_te`) con batching.  

Aquí se convierte las notas clínicas en un formato que el modelo BETO puede procesar, integrando tanto los textos tokenizados como las etiquetas de clasificación.

Además el texto clínico ya se transforma a la **entrada real** que BETO usa (ids + máscaras).

#### Cargar BERT (TF) y compilar modelo

In [ ]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

In [ ]:
model_name = "dccuchile/bert-base-spanish-wwm-uncased"

bert = TFAutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=len(labels), id2label=id2label, label2id=label2id
)

# Adam de tf.keras (NO legacy, NO keras standalone)
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)  # opcional: clipnorm=1.0

# Pérdida explícita para logits
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Métrica
metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name="accuracy")]

bert.compile(optimizer=optimizer, loss=loss, metrics=metrics)

tf_model.h5:   0%|          | 0.00/537M [00:00<?, ?B/s]

TensorFlow and JAX classes are deprecated and will be removed in Transformers v5. We recommend migrating to PyTorch classes or pinning your version of Transformers.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


> Se vuelve a declarar `model_name` en este bloque para mantener claridad y modularidad en el notebook.  



-  
  Se eligió `Adam` porque es un optimizador muy usado en *fine-tuning* de modelos grandes como BERT.  
  Combina lo mejor de *Momentum* y *Adaptive Learning Rate*, ajustando dinámicamente la tasa de aprendizaje para cada parámetro.  
  Esto es clave en transformers, ya que entrenan millones de parámetros y requieren estabilidad.  

-   
  Los modelos preentrenados como BETO ya traen mucho conocimiento adquirido.  
  Una tasa demasiado alta podría “destruir” ese conocimiento rápidamente.  
  Con `2e-5` se logra un ajuste fino y progresivo, evitando sobreajuste y manteniendo la generalización.  

-   
  - *CategoricalCrossentropy* es la pérdida estándar para clasificación multiclase.  
  - La variante *Sparse* se usa porque las etiquetas están codificadas como enteros (0,1,2) y no en formato one-hot.  
  - El argumento `from_logits=True` es necesario porque BERT entrega salidas sin normalizar (logits). Esto le indica a la función de pérdida que aplique el *softmax* internamente antes de calcular el error.  

-   
  Se eligió `accuracy` porque es una métrica clara y sencilla para el monitoreo inicial.  
  Aunque más adelante se analicen métricas más detalladas (precisión, recall, F1 por clase), `accuracy` permite tener una idea rápida de si el modelo está aprendiendo correctamente.  

---




#### EarlyStopping

In [ ]:
cb_es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=1, restore_best_weights=True
)

* Monitor a `val_los`s (es más estable que `val_accuracy` en fine-tuning).

#### Entrenar 2–3 épocas

In [ ]:
history = bert.fit(
    tf_tr,
    validation_data=tf_te,
    epochs=3,
    callbacks=[cb_es],   # early stopping
    verbose=1
)


Epoch 1/3
10/10 [==============================] - 61s 740ms/step - loss: 0.8529 - accuracy: 0.6500 - val_loss: 0.3761 - val_accuracy: 1.0000
Epoch 2/3
10/10 [==============================] - 2s 156ms/step - loss: 0.2771 - accuracy: 1.0000 - val_loss: 0.0885 - val_accuracy: 1.0000
Epoch 3/3
10/10 [==============================] - 2s 167ms/step - loss: 0.0711 - accuracy: 1.0000 - val_loss: 0.0286 - val_accuracy: 1.0000


Se entrenó 3 épocas y la **val_accuracy fue 1.000 desde la 1.ª época**, con `val_loss` cayendo de 0.376 → 0.088 → 0.029.  

El set de prueba resulta **muy fácil** para el modelo. Aun después de limpiar `afeccion`, el **`texto_clinico` trae señales fuertes de la gravedad** (p. ej., menciones explícitas “leve/moderado/severo” u otros términos altamente separables), o bien hay **textos muy similares** entre train/test.

####  Predicción y reporte (precision/recall/F1)

In [ ]:
logits_list, y_true_ids_list = [], []

for x_batch, y_batch in tf_te:            # <- desempaquetar (features, labels)
    # x_batch es un dict con 'input_ids' y 'attention_mask'
    outputs = bert(x_batch, training=False)
    logits_list.append(outputs.logits.numpy())
    y_true_ids_list.append(y_batch.numpy())

logits = np.vstack(logits_list)
y_true_ids = np.concatenate(y_true_ids_list)
y_pred_ids = logits.argmax(axis=-1)

y_true_lbls = [id2label[i] for i in y_true_ids]
y_pred_lbls = [id2label[i] for i in y_pred_ids]

print("== BERT (BETO) ==")
print(classification_report(y_true_lbls, y_pred_lbls, digits=3))
print(confusion_matrix(y_true_lbls, y_pred_lbls))

== BERT (BETO) ==
              precision    recall  f1-score   support

        leve      1.000     1.000     1.000        12
    moderado      1.000     1.000     1.000        16
      severo      1.000     1.000     1.000        12

    accuracy                          1.000        40
   macro avg      1.000     1.000     1.000        40
weighted avg      1.000     1.000     1.000        40

[[12  0  0]
 [ 0 16  0]
 [ 0  0 12]]


- **Métricas**: precisión/recall/F1 = **1.000** en **todas** las clases; matriz de confusión **diagonal perfecta** (12/16/12 por clase).  

- El set de prueba es **trivial** para el modelo. Aun sin “fuga” desde `afeccion`, el **texto_clínico** contiene señales muy fuertes (p. ej., menciones explícitas de gravedad o términos casi determinísticos por clase), o existen **textos repetidos/muy similares** entre train/test. Con 40 casos de test, además, los intervalos de confianza son amplios.

---

### PUNTO 3 — **Sesgos (estratificado)**.

En esta sección se evalúa **fairness** midiendo el desempeño en el **set de prueba** por subgrupos.

Primero, se compara género (*F vs M*): se alinean las etiquetas verdaderas y predichas con el índice de `X_test`, y se calculan **precision**, **recall** y **F1** *macro* sobre **cada subpoblación** usando máscaras booleanas. El promedio macro da el mismo peso a cada clase (*leve/moderado/severo*), evitando que una clase frecuente domine la métrica; `zero_division=0` evita errores cuando alguna clase no aparece en un subgrupo.

El resultado se presenta en una **tabla por grupo** con P/R/F1 (macro). Esta comparación permite detectar **brechas de rendimiento** entre F y M. Si aparecen diferencias relevantes (p. ej., >5–10 puntos de F1 macro), se dejan **propuestas de mitigación** (p. ej., class_weight, rebalanceo, validación estratificada, más datos). Luego se repite la misma lógica para rangos etarios (`<30`, `30–50`, `>50`).

#### Sesgos por género

In [ ]:
from sklearn.metrics import precision_recall_fscore_support


In [ ]:
y_true_s = pd.Series(y_true_lbls, index=X_test.index)
y_pred_s = pd.Series(y_pred_lbls, index=X_test.index)

def macro_by_mask(y_true, y_pred, mask):
    p,r,f1,_ = precision_recall_fscore_support(y_true[mask], y_pred[mask],
                                               average="macro", zero_division=0)
    return {"P":p, "R":r, "F1":f1}

mask_F = X_test["genero"].eq("F")
mask_M = X_test["genero"].eq("M")

res_gen = pd.DataFrame([
    {"grupo":"F", **macro_by_mask(y_true_s, y_pred_s, mask_F)},
    {"grupo":"M", **macro_by_mask(y_true_s, y_pred_s, mask_M)},
])
res_gen

,grupo,P,R,F1
0,F,1.0,1.0,1.0
1,M,1.0,1.0,1.0


Se calcularon **precision/recall/F1 macro** por subgrupo (F y M) filtrando el conjunto de prueba y evaluando SOLO dentro de cada grupo.  
El resultado muestra **1.0 en P/R/F1 para ambos géneros**, lo que indica **cero errores** en este test: no se observa brecha entre grupos.

Las métricas perfectas aquí no prueban “ausencia de sesgo”; solo reflejan un **efecto techo** (el modelo acertó todo). Para evaluar sesgo con más evidencia:
- reportar el **soporte** por grupo (cuántos casos F/M en test);
- repetir la estratificación tras una **ablación** (p. ej., eliminar “leve/moderado/severo” de `texto_clinico`) o con un test más difícil;
- enfocarse en **recall de la clase *severo*** por grupo (igualdad de oportunidad).


#### Sesgos por tramos de edad

In [ ]:
bins = pd.cut(X_test["edad"], bins=[-1,29,50,200], labels=["<30","30–50",">50"])

res_age = []
for b in ["<30","30–50",">50"]:
    m = bins==b
    res_age.append({"grupo":f"Edad {b}", **macro_by_mask(y_true_s, y_pred_s, m)})
pd.DataFrame(res_age)


,grupo,P,R,F1
0,Edad <30,1.0,1.0,1.0
1,Edad 30–50,1.0,1.0,1.0
2,Edad >50,1.0,1.0,1.0


Se evaluó **precision/recall/F1 macro** dentro de cada tramo de edad (`<30`, `30–50`, `>50`).  
El resultado es **1.0 en P/R/F1 para los tres grupos**, es decir, **no hubo errores** en el conjunto de prueba para ningún tramo.


- No se observa **brecha por edad** en este test, pero esto responde a un **efecto techo**: el modelo acertó todo (tarea muy separable y test pequeño).  
- Con métricas saturadas, **no se puede concluir ausencia de sesgo**. Falta evidencia sobre robustez.

---


#### Informe breve + mitigaciones

Se hizo la evaluación estratificada por **género** y por **tramos de edad**. En ambos casos el modelo dio **F1 = 1.0** en todos los grupos. En este test **no aparecen diferencias** entre subgrupos.

Ese 1.0 por todos lados no significa “sin sesgo”; significa **efecto techo**: el set de prueba es chico y la tarea quedó muy fácil. Incluso después de limpiar `afeccion`, el **texto clínico** probablemente trae pistas muy fuertes (palabras o patrones que “cantan” la gravedad) y puede haber textos muy parecidos entre train/test. Con ese contexto, es normal que el modelo no muestre brechas.

**Qué propondría para investigar (no lo implemento acá, queda como plan).**
1. **Ablación léxica:** en `texto_clínico`, enmascarar términos que delatan la gravedad y volver a medir F1/recall por grupo.  
2. **Revisión de duplicados/similitud** entre train y test y re-evaluar.  
3. **Validación cruzada estratificada**, reportando **soporte** por subgrupo y, en especial, **recall de *severo***.  
4. **Ablación de features demográficas:** comparar modelo **con vs. sin** género/edad; mantenerlas solo si ayudan a *severo* sin perjudicar ningún grupo.  
5. **Más y mejor dato:** ampliar y diversificar el corpus (distintos estilos de redacción, instituciones, rangos etarios).

En este set el modelo no muestra brechas, pero por **efecto techo** no puedo descartar sesgo. La recomendación es **seguir con ablaciones y validaciones estratificadas** antes de cualquier conclusión fuerte.

---


### Reflexión ética y explicabilidad

Se eligió **Random Forest + Word2Vec** como baseline robusto para un dataset pequeño y BETO para medir el techo con un transformer en español; la dupla sirve para comparar enfoques (tradicional vs. contextual) y entender qué aprende el sistema antes de pensar en uso real. Socialmente, esto implica prudencia: en clínica un falso negativo en severo no es una métrica, es un riesgo. .

El mayor riesgo es dañar a un paciente por una mala prioridad: si el modelo falla en un caso severo, puede retrasar atención. También preocupa tratar distinto a grupos (por género o edad) y terminar estigmatizando o dejando sistemáticamente atrás a alguien. Un tercer riesgo es la sobreconfianza: métricas “perfectas” invitan a creer que “ya está”, y eso puede llevar a automatizar decisiones clínicas que siempre deben pasar por un profesional. Finalmente, privacidad y consentimiento: los textos clínicos hablan de personas reales; sin buen resguardo, se pierde confianza.

Se limpió la fuente de fuga de etiqueta, se hizo evaluación estratificada y se documentó el efecto techo (las métricas perfectas no prueban ausencia de sesgo).

Próximos pasos: revisión de duplicados, validación cruzada y externa con foco en recall de severo por subgrupo, y comparar con vs. sin variables demográficas; si no aportan a seguridad, se quitan.

El sistema funciona en este corpus, pero no está listo para decisión clínica. La responsabilidad social aquí es frenar, documentar lo que sí y lo que no hace, y avanzar con ablaciones de sesgo, validación estratificada sólida y ampliación del dataset. Luego—y solo luego—se evalúa su valor práctico.

**Observación**: podrían existir pistas léxicas en texto_clinico relacionadas con la gravedad. No se implementa limpieza adicional en esta entrega, pero se propone enmascarar términos de severidad y repetir la evaluación como trabajo futuro.